# Optymalizacja i najlepsze praktyki - Demo

**Cel szkoleniowy:** Opanowanie technik optymalizacji performance'u zapytań i tabel Delta w Databricks.

**Zakres tematyczny:**
- Optymalizacja zapytań: predicate pushdown, file pruning, column pruning
- Analiza planu fizycznego (explain())
- Optymalizacja tabel: partitioning, small files problem
- Auto optimize / auto compaction
- Dobór rozmiaru plików i strategii ZORDER
- Monitoring i diagnoza performance issues

## Kontekst i wymagania

- **Dzień szkolenia**: Dzień 2 - Lakehouse & Delta Lake
- **Typ notebooka**: Demo + Workshop
- **Wymagania techniczne**:
  - Databricks Runtime 13.0+ (zalecane: 14.3 LTS)
  - Unity Catalog włączony
  - Uprawnienia: CREATE TABLE, CREATE SCHEMA, SELECT, MODIFY
  - Klaster: Standard z minimum 2 workers (zalecane: i3.xlarge lub wyżej)
- **Zależności**: Wykonany notebook 01_delta_lake_operations.ipynb

## Wstęp teoretyczny

**Cel sekcji:** Zrozumienie kluczowych mechanizmów optymalizacji w Databricks i Delta Lake.

**Typy optymalizacji:**

**1. Optymalizacja zapytań (Query Optimization):**
- **Predicate Pushdown**: Przeniesienie filtrów jak najniżej w planie wykonania
- **Column Pruning**: Odczyt tylko wymaganych kolumn (kolumnowy format Parquet)
- **File Pruning**: Ominięcie plików nieistotnych dla zapytania
- **Join Optimization**: Broadcast joins, bucket joins, sortmerge joins

**2. Optymalizacja tabel (Table Optimization):**
- **Partitioning**: Fizyczne rozdzielenie danych według kluczy
- **Z-Ordering**: Klasterowanie danych w plikach według wybranych kolumn
- **Compaction (OPTIMIZE)**: Łączenie małych plików w większe
- **Auto Compaction**: Automatyczne łączenie podczas zapisu

**3. Small Files Problem:**
Problem wydajności wywołany przez zbyt wiele małych plików w tabeli. Spark preferuje pliki 128MB-1GB dla optymalnej wydajności.

## Izolacja per użytkownik

Uruchom skrypt inicjalizacyjny dla per-user izolacji katalogów i schematów:

In [0]:
%run ../00_setup

## Konfiguracja

Import bibliotek i ustawienie zmiennych środowiskowych:

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col, lit, count, avg, sum, max, min
from pyspark.sql.types import *
import time

# Wyświetl kontekst użytkownika
print("=== Kontekst użytkownika ===")
print(f"Katalog: {CATALOG}")
print(f"Schema Bronze: {BRONZE_SCHEMA}")
print(f"Schema Silver: {SILVER_SCHEMA}")
print(f"Schema Gold: {GOLD_SCHEMA}")
print(f"Użytkownik: {raw_user}")

# Ustaw katalog i schemat jako domyślne
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {BRONZE_SCHEMA}")

# Nazwy tabel (założenie: notebook 01_delta_lake_operations został wykonany)
CUSTOMERS_DELTA = f"{BRONZE_SCHEMA}.customers_delta"
ORDERS_DELTA = f"{BRONZE_SCHEMA}.orders_delta"

print(f"\n=== Tabele do optymalizacji ===")
print(f"Customers: {CUSTOMERS_DELTA}")
print(f"Orders: {ORDERS_DELTA}")

## Sekcja 1: Analiza planu fizycznego (explain())

**Cel sekcji:** Naucz się analizować plany wykonania zapytań, aby zidentyfikować bottlenecki wydajnościowe.

**Teoria:**
Plan fizyczny to szczegółowa mapa tego, jak Spark wykonuje zapytanie:
- **Stages**: Logiczne etapy przetwarzania
- **Tasks**: Jednostki pracy wykonywane na partycjach
- **Shuffles**: Wymiana danych między executor'ami
- **Pushdowns**: Optymalizacje przeniesione do źródła danych

**Rodzaje explain():**
- `explain()` - podstawowy plan
- `explain(True)` - pełny plan z detalami
- `explain('extended')` - rozszerzony plan
- `explain('cost')` - plan z kosztem

### Przykład 1.1: Analiza planu prostego zapytania

In [0]:
# Przykład 1.1 - Analiza planu prostego zapytania

# Użyj tabeli customers_delta utworzonej w 01_delta_lake_operations.ipynb
# CUSTOMERS_DELTA jest już zdefiniowany w sekcji Konfiguracja

print("=== Podstawowy plan zapytania ===")
simple_query = spark.sql(f"""
    SELECT customer_id, first_name, last_name, customer_segment, city
    FROM {CUSTOMERS_DELTA}
    WHERE customer_segment = 'Premium'
    ORDER BY customer_id DESC
    LIMIT 10
""")

# Wyświetl podstawowy plan
simple_query.explain()

print("\n=== Rozszerzony plan zapytania ===")
# Wyświetl rozszerzony plan
simple_query.explain(True)

### Przykład 1.2: Predicate Pushdown w praktyce

**Teoria:** Predicate pushdown to optymalizacja, gdzie filtry (warunki WHERE) są "przepychane" jak najniżej w planie wykonania, najlepiej do poziomu czytania plików. Dzięki temu czytamy tylko dane, które spełniają warunki.

In [0]:
# Przykład 1.2 - Predicate Pushdown

# Użyj tabeli orders_delta utworzonej w 01_delta_lake_operations.ipynb
# ORDERS_DELTA jest już zdefiniowany w sekcji Konfiguracja

print("=== Zapytanie z filtrami (predicate pushdown) ===")
filtered_query = spark.sql(f"""
    SELECT order_id, customer_id, total_amount, order_date
    FROM {ORDERS_DELTA}
    WHERE total_amount > 100 
    AND order_date >= '2024-01-01'
""")

# Sprawdź plan - poszukaj "PushedFilters" w planie
filtered_query.explain(True)

print("\n💡 Znajdź w planie:")
print("- 'PushedFilters' - filtry przepchnięte do poziomu czytania")
print("- 'ReadSchema' - tylko wybrane kolumny (column pruning)")
print("- 'PartitionFilters' - filtry na partycjach")

## Sekcja 2: Strategia partycjonowania

**Cel sekcji:** Nauka wyboru optymalnych kluczy partycjonowania dla najlepszej wydajności.

**Teoria partycjonowania:**
- **Partitioning**: Fizyczne rozdzielenie tabeli na katalogi według wartości kolumn
- **Partition Pruning**: Spark pomija całe partycje, które nie są potrzebne dla zapytania
- **Idealne partycje**: 1-10GB danych na partycję, nie więcej niż 10,000 partycji

**Najlepsze praktyki:**
- Partycjonuj według kolumn często używanych w filtrach
- Unikaj partycjonowania według kolumn o wysokiej kardinalności
- Preferuj kolumny z naturalną hierarchią czasową (rok/miesiąc/dzień)
- Unikaj zbyt wielu małych partycji (small files problem)

### Przykład 2.1: Tworzenie tabeli partycjonowanej

In [0]:
# Przykład 2.1 - Tworzenie tabeli partycjonowanej

# Utwórz tabelę partycjonowaną według roku i miesiąca
ORDERS_PARTITIONED = f"{BRONZE_SCHEMA}.orders_partitioned"

print("=== Tworzenie tabeli partycjonowanej ===")

# Dodaj kolumny do partycjonowania
orders_with_partitions = spark.sql(f"""
    SELECT 
        *,
        YEAR(order_date) as year,
        MONTH(order_date) as month
    FROM {ORDERS_DELTA}
""")

# Zapisz jako tabelę partycjonowaną
orders_with_partitions.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("year", "month") \
    .saveAsTable(ORDERS_PARTITIONED)

print(f"✓ Tabela {ORDERS_PARTITIONED} utworzona z partycjonowaniem według roku i miesiąca")

# Sprawdź strukturę partycji
spark.sql(f"DESCRIBE DETAIL {ORDERS_PARTITIONED}").select("name", "location", "partitionColumns").show(truncate=False)

### Przykład 2.2: Partition Pruning w działaniu

In [0]:
# Przykład 2.2 - Partition Pruning

print("=== Zapytanie wykorzystujące partition pruning ===")

# Zapytanie który wykorzystuje partycje (rok/miesiąc)
efficient_query = spark.sql(f"""
    SELECT order_id, customer_id, total_amount, order_date
    FROM {ORDERS_PARTITIONED}
    WHERE year = 2024 AND month = 1
""")

# Sprawdź plan - poszukaj "PartitionFilters"
efficient_query.explain(True)

print("\n=== Porównanie: zapytanie BEZ partition pruning ===")

# Zapytanie które nie wykorzystuje partycji (nie filtruje po roku/miesiącu)
inefficient_query = spark.sql(f"""
    SELECT order_id, customer_id, total_amount, order_date
    FROM {ORDERS_PARTITIONED}
    WHERE customer_id = 1
""")

inefficient_query.explain(True)

print("\n💡 Porównaj plany:")
print("- Pierwszy: partition pruning aktywny (PartitionFilters)")
print("- Drugi: brak partition pruning - czyta wszystkie partycje")

## Sekcja 3: Small Files Problem

**Cel sekcji:** Zrozumienie i rozwiązanie problemu małych plików w Delta Lake.

**Co to jest Small Files Problem?**
- Gdy tabela ma zbyt wiele małych plików (< 128MB każdy)
- Spark preferuje pliki 128MB-1GB dla optymalnej wydajności
- Małe pliki powodują overhead w metadanych i zmniejszają throughput

**Przyczyny małych plików:**
- Częste zapisy INSERT w małych batch'ach
- Wysokie partycjonowanie z małą ilością danych na partycję
- Streaming z krótkimi trigger intervals

**Rozwiązania:**
- **OPTIMIZE** - łączy małe pliki w większe
- **Auto Compaction** - automatyczne łączenie podczas zapisu
- **Repartition** przed zapisem
- **Coalesce** dla zmniejszenia liczby partycji

### Przykład 3.1: Symulacja Small Files Problem

In [0]:
# Przykład 3.1 - Symulacja Small Files Problem

SMALL_FILES_TABLE = f"{BRONZE_SCHEMA}.small_files_demo"

print("=== Tworzenie tabeli z małymi plikami ===")

# Symuluj wiele małych zapisów (każdy tworzy osobny plik)
for i in range(5):
    small_batch = spark.range(i*100, (i+1)*100).select(
        col("id"),
        (col("id") * 2).alias("value"),
        lit(f"batch_{i}").alias("batch_name")
    )
    
    small_batch.write \
        .format("delta") \
        .mode("append") \
        .saveAsTable(SMALL_FILES_TABLE)
    
    print(f"✓ Zapisano batch {i}")

print("\n=== Sprawdzenie liczby plików ===")
detail = spark.sql(f"DESCRIBE DETAIL {SMALL_FILES_TABLE}").collect()[0]
print(f"Liczba plików: {detail['numFiles']}")
print(f"Rozmiar tabeli: {detail['sizeInBytes']} bytes")
print(f"Średni rozmiar pliku: {detail['sizeInBytes'] / detail['numFiles']:.0f} bytes")

print("\n⚠️  Problem: zbyt wiele małych plików!")

### Przykład 3.2: Rozwiązanie - OPTIMIZE i Auto Compaction

In [0]:
# Przykład 3.2 - Rozwiązanie Small Files Problem

print("=== Rozwiązanie: OPTIMIZE ===")

# Wykonaj OPTIMIZE na tabeli z małymi plikami
spark.sql(f"OPTIMIZE {SMALL_FILES_TABLE}")

# Sprawdź stan po OPTIMIZE
detail_after = spark.sql(f"DESCRIBE DETAIL {SMALL_FILES_TABLE}").collect()[0]
print(f"\nPo OPTIMIZE:")
print(f"Liczba plików: {detail_after['numFiles']}")
print(f"Rozmiar tabeli: {detail_after['sizeInBytes']} bytes")
print(f"Średni rozmiar pliku: {detail_after['sizeInBytes'] / detail_after['numFiles']:.0f} bytes")

print("\n=== Konfiguracja Auto Compaction ===")

# Włącz auto compaction dla przyszłych zapisów
spark.sql(f"""
    ALTER TABLE {SMALL_FILES_TABLE}
    SET TBLPROPERTIES (
        'delta.autoOptimize.optimizeWrite' = 'true',
        'delta.autoOptimize.autoCompact' = 'true'
    )
""")

print("✓ Auto Compaction włączony")

# Sprawdź właściwości tabeli
properties = spark.sql(f"SHOW TBLPROPERTIES {SMALL_FILES_TABLE}").collect()
auto_props = [p for p in properties if 'autoOptimize' in p['key']]
print("\nWłączone właściwości Auto Compaction:")
for prop in auto_props:
    print(f"- {prop['key']}: {prop['value']}")

## Sekcja 4: ZORDER BY - Advanced Clustering

**Cel sekcji:** Nauka wykorzystania ZORDER BY do optymalizacji zapytań z filtrami i joinami.

**Co to jest ZORDER BY?**
- Multi-dimensional clustering algorithm w Delta Lake
- Organizuje dane w plikach według wartości wybranych kolumn
- Poprawia data skipping - pomijanie niepotrzebnych plików podczas czytania
- Szczególnie skuteczny dla kolumn często używanych w filtrach WHERE i JOIN

**Kiedy używać ZORDER:**
- Kolumny często filtrowane w zapytaniach
- Kolumny używane w JOIN operations
- High-cardinality columns (wiele unikalnych wartości)
- Maksymalnie 3-4 kolumny (więcej = diminishing returns)

**ZORDER vs Partitioning:**
- Partitioning: fizyczne rozdzielenie na katalogi
- ZORDER: logiczne uporządkowanie w plikach (zachowuje pojedynczą strukturę folderów)

### Przykład 4.1: ZORDER BY dla często filtrowanych kolumn

In [0]:
# Przykład 4.1 - ZORDER BY

print("=== Wykonanie ZORDER BY ===")

# Załóżmy, że często filtrujemy podle customer_id i order_date
# Wykonaj ZORDER BY na tych kolumnach
spark.sql(f"""
    OPTIMIZE {ORDERS_DELTA}
    ZORDER BY (customer_id, order_date)
""")

print("✓ ZORDER BY wykonany na kolumnach: customer_id, order_date")

# Sprawdź historię optymalizacji
print("\n=== Historia operacji OPTIMIZE ===")
history = spark.sql(f"DESCRIBE HISTORY {ORDERS_DELTA}")
optimize_operations = history.filter(col("operation") == "OPTIMIZE")

display(
    optimize_operations.select(
        "version", "timestamp", "operation", 
        "operationParameters", "operationMetrics"
    ).orderBy("version", ascending=False).limit(3)
)

print("\n💡 Sprawdź 'operationParameters' - powinno zawierać informację o ZORDER")

### Przykład 4.2: Pomiar skuteczności ZORDER - Data Skipping

In [0]:
# Przykład 4.2 - Pomiar skuteczności ZORDER

import time

print("=== Test skuteczności ZORDER BY ===")

# Zapytanie wykorzystujące kolumny z ZORDER
test_query = spark.sql(f"""
    SELECT COUNT(*), AVG(total_amount)
    FROM {ORDERS_DELTA}
    WHERE customer_id BETWEEN 100 AND 200
    AND order_date >= '2024-01-01'
""")

# Pomiar czasu wykonania
start_time = time.time()
result = test_query.collect()
end_time = time.time()

print(f"Wynik: {result[0]}")
print(f"Czas wykonania: {end_time - start_time:.2f} sekund")

# Sprawdź plan zapytania - data skipping
print("\n=== Plan zapytania (sprawdź data skipping) ===")
test_query.explain(True)

print("\n💡 W planie szukaj:")
print("- 'numFilesTotal' vs 'numFilesSelected' - ile plików pominięto")
print("- 'metadata time' - czas parsowania metadanych")
print("- 'files pruned' - data skipping statistics")

## Best Practices - Przewodnik optymalizacji

### 🎯 Strategia optymalizacji (w kolejności priorytetów):

**1. Analiza workload:**
- Zidentyfikuj najczęściej wykonywane zapytania
- Znajdź kolumny najczęściej używane w filtrach WHERE
- Sprawdź które zapytania zajmują najwięcej czasu

**2. Optymalizacja zapytań:**
- Używaj filtrów WHERE jak najwcześniej w zapytaniu
- Wybieraj tylko potrzebne kolumny (SELECT specific columns, nie *)
- Preferuj JOIN na zindeksowanych kolumnach
- Używaj LIMIT gdy możliwe

**3. Optymalizacja tabel:**
- **Partitioning**: Tylko dla dużych tabel (>1TB) i często filtrowanych kolumn
- **ZORDER BY**: Dla 2-4 najczęściej filtrowanych kolumn
- **OPTIMIZE**: Regularnie (np. daily) dla aktywnych tabel
- **Auto Compaction**: Włącz dla tabel z częstymi zapisami

**4. Monitoring i maintenance:**
- Regularnie sprawdzaj `DESCRIBE DETAIL` - liczba plików, rozmiar
- Uruchamiaj `VACUUM` co najmniej raz w tygodniu
- Monitoruj Spark UI dla długo działających zapytań
- Używaj `explain()` do analizy problemowych zapytań

## Troubleshooting - Diagnoza problemów wydajnościowych

### 🔍 Najczęstsze problemy i rozwiązania:

**Problem 1: Zapytanie działa bardzo wolno**
```python
# Diagnoza:
df.explain(True)  # Sprawdź plan wykonania
```
**Możliwe przyczyny:**
- Brak filtrów - czyta całą tabelę
- Shuffle operations - dużo ruchu sieciowego  
- Skewed data - nierównomierne rozłożenie danych
- Small files - zbyt wiele małych plików

**Problem 2: "OutOfMemoryError" podczas JOIN**
**Rozwiązanie:**
```python
# Zwiększ partycje przed JOIN
df1 = df1.repartition(200, "join_key")
df2 = df2.repartition(200, "join_key")

# Lub użyj broadcast join dla małych tabel
from pyspark.sql.functions import broadcast
result = large_df.join(broadcast(small_df), "key")
```

**Problem 3: Długie czasy zapisu do Delta**
**Rozwiązanie:**
- Włącz Auto Compaction
- Użyj `coalesce()` przed zapisem
- Avoid zbyt wysokie partycjonowanie

**Problem 4: OPTIMIZE nie poprawia wydajności**
**Przyczyna:** ZORDER BY jest potrzebny dla specific query patterns
```python
# Zamiast samego OPTIMIZE:
OPTIMIZE table_name

# Użyj OPTIMIZE z ZORDER:
OPTIMIZE table_name ZORDER BY (frequently_filtered_columns)
```

## Podsumowanie

### ✅ W tym notebooku nauczyliśmy się:

**Analiza wydajności:**
- Czytanie i interpretacja planów fizycznych z `explain()`
- Identyfikacja bottlenecków: shuffles, skewed data, small files
- Wykorzystanie predicate pushdown i column pruning

**Optymalizacja tabel:**
- Strategia partycjonowania według często filtrowanych kolumn
- ZORDER BY dla multi-dimensional clustering
- Rozwiązywanie Small Files Problem przez OPTIMIZE
- Auto Compaction dla automatycznej optymalizacji

**Monitoring i maintenance:**
- Regularne wykonywanie DESCRIBE DETAIL
- Strategia VACUUM dla storage management
- Pomiar skuteczności optymalizacji

### 🎯 Kluczowe wnioski:

1. **Analiza przed optymalizacją**: Zawsze najpierw zdiagnozuj problem przez `explain()`
2. **Partitioning ≠ ZORDER**: Różne techniki dla różnych przypadków użycia
3. **Small files = performance killer**: Regularne OPTIMIZE jest konieczne
4. **ZORDER BY**: Maksymalnie 3-4 kolumny, wybieraj najczęściej filtrowane
5. **Auto Compaction**: Must-have dla production workloads

### 📈 Metryki do monitorowania:

| Metryka | Dobra wartość | Akcja jeśli przekroczona |
|---------|---------------|--------------------------|
| Liczba plików | < 1000/TB | OPTIMIZE |
| Średni rozmiar pliku | 128MB-1GB | OPTIMIZE + Auto Compaction |
| Czas zapytania | Baseline +20% | Analiza explain(), ZORDER |
| Skipped files ratio | >80% | ZORDER BY na filtrowanych kolumnach |

### 🚀 Następne kroki:
- **02_medallion_architecture.ipynb**: Architektura Bronze/Silver/Gold
- **03_batch_streaming_load.ipynb**: Copy Into, Auto Loader, Streaming
- **04_bronze_silver_gold_pipeline.ipynb**: End-to-end data pipeline

## Cleanup

Opcjonalnie usuń tabele demo utworzone podczas ćwiczeń:

In [0]:
# Cleanup - usuń tabele demo

# Odkomentuj poniższe linie aby usunąć tabele demo:

# spark.sql(f"DROP TABLE IF EXISTS {ORDERS_PARTITIONED}")
# spark.sql(f"DROP TABLE IF EXISTS {SMALL_FILES_TABLE}")

# print("✓ Tabele demo usunięte")

print("Cleanup wyłączony (odkomentuj kod aby usunąć tabele demo)")